In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'animal-detection-small-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F37898%2F57718%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T141102Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3fb75911db301ed36252a0cd5645226a6c81b6b9b99bfd4e928613bfba2ef4ab2141795c3348b5c003c2b72cc940a8b04538555e73265dcb599cc7a1367c52879afa49972634fc0ec9b44076c1f2184deba23ed158b3036ed718a8d8200e1daf0020a98cd424b22680be6c45cba85acaef9c1dddf93607af4fbf67bd86c8e2b2fe5a49e330949b6dc022c1367b0203abee8a848f37beda132a16a5139397934ef671ea7216cd708c4e4185b03bf740b31cdeaf63350e7bd9518f85e8618a430d8ecc34acaf10474e545d9ce64ed2b84f89b037e0f2db0ef0fbb970986304b1404709fa24f23225fb885881c57a5928ac1e57702e3ecdec8e5850135de5d56d5b,animal-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F616920%2F1102439%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T141102Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dd4a099786465bb39c38d4daf5bdc3f6698532c8a0b6d51a8d197a0eb3e9ef5a474f90c3d37e6c9c9ae278d5d3810813f48a245123ef670ed966bb114e6cf120cdb1fbb6c6e89e4efedeab5a54c18b75a18b64acdad7bebeda07157e3fed646d870a133edaa7e815fab3e4e4c397dd2091b60a66b63cfb3da43ae5803582489b381804a2284c0898cc6c8f0d1ab381e01d3bfff644d010c3b87e0c17c02b11859fa5957ba43862f5f93c7543ae527f81cf63a44898e3e881149ca66565a74604d2b4748724bac38b1dd351581a69c4cfcbe8b5eceb8d930d01efe37c01818ac59370f169d02de8055cf7a1b9ecbbf1bff2bb41b051060ac032836cd996deec87f,african-wildlife:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F674157%2F1185810%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T141102Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dbeb365d12d7f56e230abdcfc331a9837b4dba8fe1978d7d0090d9f149d649ac9bd25c33890df99129c3ccba78dd817596ef17a4046aec8f04aaefe18efc1d6234cc03385ea71faa8d1b743444b2a48731fa74b4721bd484e738c66cb731f610001421cc87ccb38adc48f73619284ea0386e335d67155c203b0c579f982af6104dc664babbc8cdfc328f4ac76e27123379503e3ac43a7d6733d9d81b3a759fbe486846444b642d76407d36dd5ee67df9cf32b81f01627a95fde3132f247e8030dac2fa7fc838802316459c19a27cc84039aa805d5e0481f43cc62bda072ecfb3ee3a2c7878f222fcdfdb4586e2e754ca4cb554ac7a910768e7fe08a72a3e7e9e4,cheetahtigerwolf:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F731756%2F1270998%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T141102Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D17b311113139f260e0d7a80b96a0c14abc02fd70f4220276b83c31bf99cfdea52deca25fc829d4d60265f9dc56d99893d49cac9a302bbe9f337e9e3e44da932f73bb9d5c0eed5d14c1038f514f217ecd378dc31d090e4f2fe53def6ead697ceb3b14dd085073aec1fc2e32c54ed17b573e7d127e5c50352146f1658dad800ab330f8c9d91d7f0b3611da372cb41b46400c46f172832d8905f691c3990730abe81ea809b116f3a0280afb8f7b4a3376523f1248605863717a6db2505085ef44ca678fe051ec727b606ce8d9e10686c5a97fd03fe7ae8e0c09b0662862e5252fc6dccf204483428e3da72dfe2e428205d606da54f9b7497bdc501c94cbdf22b304'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


## Task 1: Importing Libraries

In [ ]:
import keras
import os
import shutil
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from platform import python_version

print('Python version:', python_version())
print('Numpy version:', np.__version__)
print('Seaborn version:', sns.__version__)
from distutils.dir_util import copy_tree
import tensorflow as tf
print('tensorflow version: ',tf.__version__)
print('keras version:', keras.__version__)

In [ ]:
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout
from keras_preprocessing.image import ImageDataGenerator

## Task 2: Dataset Creation

In [ ]:
#created data set using console
source='../input/african-wildlife/'
target='./train_data/'
shutil.copytree(source, target)
os.mkdir('test_data')

In [ ]:
# remove unwanted data and create same classed for test_data

path="./train_data/"
for file in os.listdir(path):
    for image in os.listdir(path+file+'/'):
        if '.jpg' not in image:
            os.remove(path+file+'/'+image)
    os.mkdir('./test_data/'+file)


In [ ]:
# create test_data by taking 25% images from data

total_train_images,total_test_images,total_train_classes,total_test_classes=0,0,0,0
path="./train_data/"
for file in os.listdir(path):
    total_train_classes+=1
    total_images=len(os.listdir(path+file+"/"))
    test_image_count=(25/100)*total_images #25% for test and 75% for train
    for i in range(math.ceil(test_image_count)):
        img=random.choice(os.listdir(path+file+'/'))
        shutil.move(path+file+'/'+img,'./test_data/'+file+'/')
        #print(img)
    print(file,total_images,math.ceil(test_image_count))
    total_train_images+=(total_images-math.ceil(test_image_count))
    #print(file,math.ceil(test_image_count))
print("total train images are : ",total_train_images," and total train classes are : ",total_train_classes)

## Task 3: Model Creation

In [ ]:
model = Sequential()

In [ ]:
#inputlayer : apply filters
model.add(Convolution2D(filters=32,
                        kernel_size=(3,3),
                        strides=(1,1),
                        padding='same',
                        activation='relu',
                   input_shape=(32, 32, 1)))

In [ ]:
# pooling layer where we are doing maxpooling
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
#adding one more convolution layer for better model
model.add(Convolution2D(filters=32,
                        kernel_size=(3,3),
                        strides=(1,1),
                        padding='same',
                        activation='relu'
                      ))

In [ ]:
#adding one more Pooling layer for better model
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
#dropout regularlization
model.add(Dropout(0.5))

In [ ]:
#layer in which we are converting 2d/3d image to 1d image i.e flattening
model.add(Flatten())

In [ ]:
# layer: appling relu to give positive output from here our hidden layerrs starts
model.add(Dense(units=20, activation='relu'))

In [ ]:
#dropout regularlization
model.add(Dropout(0.5))

In [ ]:
# output layer : Since we have to do multi-class classification so we'll apply softmax activation function
# we have 4 classes of animals so output layer would have that many neurons.
model.add(Dense(units=4, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

## Task 4: Image Augmentation

In [ ]:
#url : https://keras.io/api/preprocessing/image/
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        './train_data/',
        target_size=(32,32),
        color_mode="grayscale",
        batch_size=64,
        class_mode='categorical')
test_set = test_datagen.flow_from_directory(
        './test_data/',
        target_size=(32,32),
        color_mode="grayscale",
        batch_size=64,
        class_mode='categorical')

In [ ]:
training_set.class_indices # to see classes of our dataset

## Task 5: Model Training

In [ ]:
history = model.fit(
        training_set,
        steps_per_epoch=(1125/64),
        epochs=100,
        validation_data=test_set,
        validation_steps=(376/64))

## Task 6: Accuracy

In [ ]:
#Graphing our training and validation
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'r', label='Training acc')
plt.plot(epochs, val_accuracy, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
#model.save("simple_animal_classification_model.h5")#save model

In [ ]:
#from keras.models import load_model
#model=load_model("simple_animal_classification_model.h5")

## Task 7: Testing

In [ ]:
from keras.preprocessing import image
test_image = image.load_img("../input/african-wildlife/zebra/001.jpg",target_size=(32,32),color_mode='grayscale')
test_image


In [ ]:
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)

In [ ]:
result = model.predict(test_image)

my_dict=training_set.class_indices
def get_key(val):
    for key, value in my_dict.items():
         if val == value:
             return key

    return "key doesn't exist"

pred=list(result[0])
for i in range(len(pred)):
    if pred[i]!=0:
        print(get_key(i))